In [ ]:

#optical flow
import numpy as np

#!pip install opencv-python
import cv2
from matplotlib import pyplot as plt
cap = cv2.VideoCapture("carrace2.mp4") # load video file
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

while cap.isOpened():
    ret, curr_frame = cap.read()
    if not ret:
        break

    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    
    #Lucas-Kanade method
    flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    dx = np.mean(flow[..., 0])  # Horizontal motion
    dy = np.mean(flow[..., 1])  # Vertical motion

    # visual representation of motion vectors
    motion_visualization = curr_frame.copy()
    h, w = curr_gray.shape
    for y in range(0, h, 20):
        for x in range(0, w, 20):
            fx, fy = flow[y, x]
            cv2.arrowedLine(motion_visualization, (x, y), (int(x+fx), int(y+fy)), (0, 0, 255), 1)

    cv2.imshow("Motion Vectors", motion_visualization)
    
    prev_gray = curr_gray.copy()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
#motion estimation



0: 192x640 6 cars, 77.8ms
Speed: 6.3ms preprocess, 77.8ms inference, 208.5ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 6 cars, 12.8ms
Speed: 1.3ms preprocess, 12.8ms inference, 1.5ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 6 cars, 13.7ms
Speed: 1.1ms preprocess, 13.7ms inference, 1.7ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 6 cars, 18.5ms
Speed: 1.1ms preprocess, 18.5ms inference, 2.3ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 7 cars, 18.5ms
Speed: 1.2ms preprocess, 18.5ms inference, 2.3ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 6 cars, 23.0ms
Speed: 1.3ms preprocess, 23.0ms inference, 2.6ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 6 cars, 24.0ms
Speed: 1.6ms preprocess, 24.0ms inference, 2.6ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 6 cars, 29.6ms
Speed: 1.6ms preprocess, 29.6ms inference, 2.9ms postprocess per image at shape (1, 3, 192, 640)

0: 19

In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort


model = YOLO("yolov8m.pt")  # Load the YOLO model


deep_sort_tracker = DeepSort(max_age=20)# age of frame 20

# Open video file
cap = cv2.VideoCapture("carrace2.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Exit (to stop the video)

    height, width, _ = frame.shape
    top_half = frame[:height // 2, :]  # Crop the top 50% as to expell the car interiors for false detection

    
    results = model(top_half)# YOLOv8 model  on cropped video

    detections = []# to store the detected objects
    for result in results:# iterate detection result
        for box, conf in zip(result.boxes.xyxy, result.boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            w, h = x2 - x1, y2 - y1
            detections.append([[x1, y1, w, h], conf.item(), 0])  # Use width & height

    
    tracked_objects = deep_sort_tracker.update_tracks(detections, frame=top_half)# update with current frame

    # Draw detections and trackers on top half
    for track in tracked_objects:
        if not track.is_confirmed():
            continue
        x1, y1, x2, y2 = map(int, track.to_tlbr())
        track_id = track.track_id
        cv2.rectangle(top_half, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green for accurate fit
        cv2.putText(top_half, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display only the top half
    cv2.imshow("Processed (Top Half Only)", top_half)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Load YOLO Model
model = YOLO("yolov8m.pt")  # Load the YOLO model


deep_sort_tracker = DeepSort(max_age=20)

cap = cv2.VideoCapture("carrace.mp4")

# Parameters for optical flow
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Read the first frame
ret, old_frame = cap.read()
if not ret:
    print("Failed to read video")
    exit()

height, width, _ = old_frame.shape
old_top_half = old_frame[:height // 2, :]
old_gray = cv2.cvtColor(old_top_half, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Exit loop if no frame is captured

    height, width, _ = frame.shape
    top_half = frame[:height // 2, :]
    gray = cv2.cvtColor(top_half, cv2.COLOR_BGR2GRAY)

    # Calculate Optical Flow
    if p0 is not None:
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Draw optical flow vectors
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            top_half = cv2.arrowedLine(top_half, (int(c), int(d)), (int(a), int(b)), (0, 0, 255), 2)

        # Compute transformation matrix
        if len(good_new) > 0:
            transform_matrix, _ = cv2.estimateAffinePartial2D(good_old, good_new)
            if transform_matrix is not None:
                top_half = cv2.warpAffine(top_half, transform_matrix, (width, height // 2))

    # Run YOLO on the stabilized top half
    results = model(top_half)

    detections = []
    for result in results:
        for box, conf in zip(result.boxes.xyxy, result.boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            w, h = x2 - x1, y2 - y1
            detections.append([[x1, y1, w, h], conf.item(), 0])  # Use width & height

    # Update DeepSORT tracker
    tracked_objects = deep_sort_tracker.update_tracks(detections, frame=top_half)

    # Draw detections and trackers on top half
    for track in tracked_objects:
        if not track.is_confirmed():
            continue
        x1, y1, x2, y2 = map(int, track.to_tlbr())
        track_id = track.track_id
        print(f"Tracked Object {track_id}: x1={x1}, y1={y1}, x2={x2}, y2={y2}")  # Print coordinates
        cv2.rectangle(top_half, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green for accurate fit
        cv2.putText(top_half, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Update previous frame and points
    old_gray = gray.copy()
    p0 = good_new.reshape(-1, 1, 2) if len(good_new) > 0 else None

    # Display only the top half
    cv2.imshow("Processed (Top Half Only)", top_half)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()



0: 192x640 2 cars, 102.7ms
Speed: 3.6ms preprocess, 102.7ms inference, 114.0ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 1 car, 14.4ms
Speed: 1.6ms preprocess, 14.4ms inference, 2.5ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 1 car, 14.2ms
Speed: 1.1ms preprocess, 14.2ms inference, 1.7ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 (no detections), 15.0ms
Speed: 1.1ms preprocess, 15.0ms inference, 1.0ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 1 person, 16.8ms
Speed: 1.1ms preprocess, 16.8ms inference, 1.9ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 1 car, 12.8ms
Speed: 0.9ms preprocess, 12.8ms inference, 1.8ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 1 car, 13.1ms
Speed: 1.2ms preprocess, 13.1ms inference, 2.0ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 2 cars, 16.4ms
Speed: 1.1ms preprocess, 16.4ms inference, 2.4ms postprocess per image at shape (1, 3, 192, 64